In [1]:
import numpy as np
import mlflow
import mlflow.metrics

In [2]:
# Function to parse the tracking files
def parse_tracking_file(file_path):
    """
    Parses the tracking result file and returns a dictionary with frame_id as key
    and a list of detections in the format (track_id, x, y, w, h)
    """
    tracking_data = {}
    with open(file_path, "r") as f:
        for line in f:
            parts = line.strip().split(",")
            frame_id = int(parts[0])
            track_id = int(parts[1])
            x, y, w, h = map(int, parts[2:6])
            if frame_id not in tracking_data:
                tracking_data[frame_id] = []
            tracking_data[frame_id].append((track_id, x, y, w, h))
    return tracking_data

# Function to parse the ground truth file
def parse_gt_file(gt_file_path):
    """
    Parses the ground truth file and returns a dictionary with frame_id as key
    and a list of GT boxes in the format (track_id, x, y, w, h)
    """
    gt_data = {}
    with open(gt_file_path, "r") as f:
        for line in f:
            parts = line.strip().split(",")
            frame_id = int(parts[0])
            track_id = int(parts[1])
            x, y, w, h = map(int, parts[2:6])
            if frame_id not in gt_data:
                gt_data[frame_id] = []
            gt_data[frame_id].append((track_id, x, y, w, h))
    return gt_data

# IoU calculation function
def iou(bb1, bb2):
    """
    Compute IoU (Intersection over Union) between two bounding boxes.
    Each box is in the format (x, y, w, h)
    """
    x1, y1, w1, h1 = bb1
    x2, y2, w2, h2 = bb2

    xi1 = max(x1, x2)
    yi1 = max(y1, y2)
    xi2 = min(x1 + w1, x2 + w2)
    yi2 = min(y1 + h1, y2 + h2)

    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    area1 = w1 * h1
    area2 = w2 * h2
    union_area = area1 + area2 - inter_area

    return inter_area / union_area if union_area != 0 else 0

# Evaluation function for tracking
def evaluate_tracking(gt_data, pred_data, iou_threshold=0.5):
    """
    Evaluate tracking performance based on IoU threshold.
    Returns MOTA, IDF1 scores.
    """
    tp = 0  # True positives
    fp = 0  # False positives
    fn = 0  # False negatives
    num_tracks = 0  # Total number of tracks

    for frame_id in gt_data:
        if frame_id not in pred_data:
            continue

        gt_tracks = gt_data[frame_id]
        pred_tracks = pred_data[frame_id]

        # Match predicted tracks with ground truth
        matched_gt = set()
        matched_pred = set()

        for pred in pred_tracks:
            best_iou = 0
            best_gt = None
            for gt in gt_tracks:
                if gt[0] in matched_gt:  # Skip already matched GTs
                    continue
                iou_score = iou(pred[1:], gt[1:])
                if iou_score > best_iou:
                    best_iou = iou_score
                    best_gt = gt

            if best_iou >= iou_threshold:
                tp += 1
                matched_gt.add(best_gt[0])
                matched_pred.add(pred[0])
            else:
                fp += 1

        fn += len(gt_tracks) - len(matched_gt)
        num_tracks += len(gt_tracks)

    # Compute evaluation metrics
    mota = 1 - (fp + fn) / num_tracks if num_tracks > 0 else 0
    idf1 = 2 * tp / (2 * tp + fp + fn) if (2 * tp + fp + fn) > 0 else 0

    return mota, idf1

In [3]:
# Set MLflow experiment name (you can customize this)
mlflow.set_experiment('Tracking_Evaluation_Experiment')

gt_file_path = "tracking_rukomet/tracks_gt/DSC_2411_trk-ispravljen.txt"
pred_deepsort_file = "tracking_rukomet/predictions/DSC_2411_deepsort.txt"
pred_norfair_file = "tracking_rukomet/predictions/DSC_2411_norfair.txt"

# Parse ground truth and predictions
gt_data = parse_gt_file(gt_file_path)
pred_deepsort_data = parse_tracking_file(pred_deepsort_file)
pred_norfair_data = parse_tracking_file(pred_norfair_file)

# MLflow run
with mlflow.start_run():
    # Log parameters (you can log the IoU threshold as well)
    mlflow.log_param("iou_threshold", 0.5)

    # Evaluate DeepSORT predictions
    mota_deepsort, idf1_deepsort = evaluate_tracking(gt_data, pred_deepsort_data)
    mlflow.log_metric("MOTA_DeepSORT", mota_deepsort)
    mlflow.log_metric("IDF1_DeepSORT", idf1_deepsort)

    # Evaluate Norfair predictions
    mota_norfair, idf1_norfair = evaluate_tracking(gt_data, pred_norfair_data)
    mlflow.log_metric("MOTA_Norfair", mota_norfair)
    mlflow.log_metric("IDF1_Norfair", idf1_norfair)

    # End the MLflow run
    mlflow.end_run()

2025/01/30 21:53:06 INFO mlflow.tracking.fluent: Experiment with name 'Tracking_Evaluation_Experiment' does not exist. Creating a new experiment.
